In [ ]:
print("Importing packages...")
import cv2
import numpy as np
import torch
import time
from IPython.display import display, clear_output
from PIL import Image
import io
import gym
import random
import gym_donkeycar
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from stable_baselines3 import PPO
print("Packages imported.")

In [ ]:
print("Loading model...")
try:
    model = PPO.load("model_edecay_98305_robert.zip")
except KeyError:
    raise Exception("ModelError: Model trained in Stable-Baselines2, but trying to load using Stable-Baselines3.")
print("Model loaded!")

print("Initializing car...")
car = NvidiaRacecar()
print("Car initialized!")

print("Initializing video capture...")
camera = CSICamera(width=224, height=224, capture_fps=65)
print("Camera ready.")

In [ ]:
def preprocess_image(image, crop_height=135):
    cropped = image[-crop_height:, :, :]
    image = cropped / 255.0
    return image.transpose(2, 0, 1)  # CHW format


In [ ]:
print("Starting program...")
ENGAGE_MOTOR = True
STEERING_GAIN = 0.7
STEERING_PROBABILITY = 1
EXTRA_CONSTANT_SPEED = 0.2
SPEED_LIMIT = 0.3
print(f"ENGAGE_MOTOR: {ENGAGE_MOTOR} | STEERING_PROBABILITY: {STEERING_PROBABILITY} | STEERING_GAIN: {STEERING_GAIN} | EXTRA_CONSTANT_SPEED: {EXTRA_CONSTANT_SPEED} | SPEED_LIMIT: {SPEED_LIMIT}")

print("Warming up camera...")


try:
    while True:
        image = camera.read()
        
        input_tensor = preprocess_image(image)
        
        print(input_tensor.shape)

        # Run model inference
        with torch.no_grad():
            action, _states = model.predict(input_tensor)
        print(action)
        # Convert action to steering and throttle values
        steering = float(action[0])
        throttle = float(action[1])
        print(f"Steering: {steering}, Throttle: {-throttle}")
        
        # Apply controls to the car
        if ENGAGE_MOTOR:
            if random.random() > 1-STEERING_PROBABILITY:
                car.steering = -steering * STEERING_GAIN  # Steering controls might be reversed?
            else:
                car.steering = 0
            
            if throttle >= 0:
                throttle = min(throttle, SPEED_LIMIT)  # Limit speed
                throttle += EXTRA_CONSTANT_SPEED       # Add constant speed
            car.throttle = -throttle                   # Throttle controls are inverted
        
        # Display the camera feed and preprocessed feed inline in the notebook
        rgb_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb_frame).resize((224, 224))
        
        buf = io.BytesIO()
        pil_img.save(buf, format='PNG')
        buf.seek(0)

        clear_output(wait=True)
        display(Image.open(buf))
                
except KeyboardInterrupt:
    print("Exiting...")
finally:
    # Release resources
    car.throttle = 0
    car.steering = 0
    print("Resources released and program exited.")
    
    